# Extracting Information from Text

## Information Extraction

Information can be extracted from structured or unstructured data. In structured data, there is a predictable organization of entities and relationships. 

Let's a look at an example of extracting information related to companies and their locations, if the data was stored in Python as a list of tuples `(entity, relation, entity)`. Using this information, we'll try to answer the question "Which organizations operate in Atlanta?"

In [1]:
import nltk
import re
import pprint
from IPython.display import display

In [2]:
locs = [('Omnicom', 'IN', 'New York'),
         ('DDB Needham', 'IN', 'New York'),
         ('Kaplan Thaler Group', 'IN', 'New York'),
         ('BBDO South', 'IN', 'Atlanta'),
         ('Georgia-Pacific', 'IN', 'Atlanta')]

In [3]:
query = [e1 for (e1, rel, e2) in locs if e2=='Atlanta']
print(query)

['BBDO South', 'Georgia-Pacific']


Over the next sections of this notebook, we'll follow a process of processing raw text to convert it into a more structured form so that information extraction is possible.

The first three steps of this task are segmentation, tokenization, and part-of-speech tagging. 

Let's define a function to do those.

In [4]:
def ie_preprocess(document):
    sentences = nltk.sent_tokenize(document)
    sentences = [nltk.word_tokenize(sent) for sent in sentences]
    sentences = [nltk.pos_tag(sent) for sent in sentences]

## Chunking

Chunking is a technique for entity detection, which segments and labels multi-token sequences.

### Noun Phrase Chunking

In Noun Phrase chunking (NP chunking), we search for chucks corresponding to individual noun phrases. 

Let's construct a simple NP-chunker by defining a chunk grammar.

In [5]:
sentence = [("the", "DT"), ("little", "JJ"), ("yellow", "JJ"), ("dog", "NN"), ("barked", "VBD"), ("at", "IN"),  ("the", "DT"), ("cat", "NN")]

In [6]:
grammar = "NP: {<DT>?<JJ>*<NN>}"

In [7]:
cp = nltk.RegexpParser(grammar)
result = cp.parse(sentence)

In [8]:
print(result)

(S
  (NP the/DT little/JJ yellow/JJ dog/NN)
  barked/VBD
  at/IN
  (NP the/DT cat/NN))


In [9]:
# Uncomment below to draw the chunked tree
# result.draw()

### Chunking with Regular Expressions

We can build a chunk grammar consisting of multiple regex rules (or tag patterns).

In [11]:
grammar = r"""
    NP: {<DT|PP\$>?<JJ>*<NN>}
        {<NNP>+}
"""
cp = nltk.RegexpParser(grammar)
sentence = [("Rapunzel", "NNP"), ("let", "VBD"), ("down", "RP"), ("her", "PP$"), ("long", "JJ"), ("golden", "JJ"), ("hair", "NN")]

In [12]:
print(cp.parse(sentence))

(S
  (NP Rapunzel/NNP)
  let/VBD
  down/RP
  (NP her/PP$ long/JJ golden/JJ hair/NN))


Note that if a tag pattern matches at overlapping locations, the leftmost match takes precedence.

In [13]:
nouns = [("money", "NN"), ("market", "NN"), ("fund", "NN")]
grammar = "NP: {<NN><NN>}  # Chunk two consecutive nouns"
cp = nltk.RegexpParser(grammar)
print(cp.parse(nouns))

(S (NP money/NN market/NN) fund/NN)


### Exploring Text Corpora

We can using chunking to obtain phrases matching a particular sequence of POS tags from a corpus as follows:

In [30]:
cp = nltk.RegexpParser('CHUNK: {<V.*> <TO> <V.*>}')
brown = nltk.corpus.brown
for sent in brown.tagged_sents()[:100]: # only searching the first 100 sentences
    tree = cp.parse(sent)
    for subtree in tree.subtrees():
        if subtree.label() == "CHUNK":
            print(subtree)

(CHUNK combined/VBN to/TO achieve/VB)
(CHUNK continue/VB to/TO place/VB)
(CHUNK serve/VB to/TO protect/VB)
(CHUNK wanted/VBD to/TO wait/VB)
(CHUNK allowed/VBN to/TO place/VB)
(CHUNK expected/VBN to/TO become/VB)
(CHUNK expected/VBN to/TO approve/VB)
(CHUNK expected/VBN to/TO make/VB)
(CHUNK intends/VBZ to/TO make/VB)
(CHUNK seek/VB to/TO set/VB)
(CHUNK like/VB to/TO see/VB)


### Chinking

A chink is defined as a sequence of tokens that is not included in a chunk. Chinking is the process of removing a sequence of tokens from a chunk. 

Let's take a look at an example of putting an entire sentence into a single chunk, and then excluding the chinks.

In [31]:
grammar = r"""
    NP: 
        {<.*>+} # Chunk everything
        }<VBD|IN>+{ # Chink sequences of VBD and IN
"""
sentence = [("the", "DT"), ("little", "JJ"), ("yellow", "JJ"),
       ("dog", "NN"), ("barked", "VBD"), ("at", "IN"),  ("the", "DT"), ("cat", "NN")]
cp = nltk.RegexpParser(grammar)

In [32]:
print(cp.parse(sentence))

(S
  (NP the/DT little/JJ yellow/JJ dog/NN)
  barked/VBD
  at/IN
  (NP the/DT cat/NN))


## Developing and Evaluating Chunkers

### Reading IOB Format and the CONLL 2000 Corpus

Let's explore a chunked sentence from the CoNLL 2000 Corpus:

In [33]:
from nltk.corpus import conll2000
print(conll2000.chunked_sents('train.txt')[99])

(S
  (PP Over/IN)
  (NP a/DT cup/NN)
  (PP of/IN)
  (NP coffee/NN)
  ,/,
  (NP Mr./NNP Stone/NNP)
  (VP told/VBD)
  (NP his/PRP$ story/NN)
  ./.)


The corpus contains 3 chunk types: NP chunks, VP chunks and PP chunks. We can use the `chunk_types` argument to filter only the NP chunks.

In [34]:
print(conll2000.chunked_sents('train.txt', chunk_types=['NP'])[99])

(S
  Over/IN
  (NP a/DT cup/NN)
  of/IN
  (NP coffee/NN)
  ,/,
  (NP Mr./NNP Stone/NNP)
  told/VBD
  (NP his/PRP$ story/NN)
  ./.)


### Simple Evaluation and Baselines

Let's start off by establishing a baselien with a trivial chunk parser that creates no chunks:

In [36]:
from nltk.corpus import conll2000
cp = nltk.RegexpParser("")
test_sents = conll2000.chunked_sents('test.txt', chunk_types=['NP'])
print(cp.evaluate(test_sents))

ChunkParse score:
    IOB Accuracy:  43.4%%
    Precision:      0.0%%
    Recall:         0.0%%
    F-Measure:      0.0%%


The IOB tag accuracy indicates that more than a third of the words are tagged with O, i.e. not in an NP chunk. However, since our tagger did not find any chunks, its precision, recall, and f-measure are all zero.

Let's improve upon our chunker and implement a tag pattern that looks for tags beginning with letters that are characteristic of noun phrase tags (ex. CD, DT, and JJ).

In [39]:
grammar = r"NP: {<[CDJNP].*>+}"
cp = nltk.RegexpParser(grammar)
print(cp.evaluate(test_sents))

ChunkParse score:
    IOB Accuracy:  87.7%%
    Precision:     70.6%%
    Recall:        67.8%%
    F-Measure:     69.2%%


**Data-driven Chunker**

We can improve upon our chunker by using a data driven approach. Let's use our training corpus to implement a unigram tagger that finds the most likely chunk tag (I, O or B) for each POS tag. 

In [41]:
class UnigramChunker(nltk.ChunkParserI):
    def __init__(self, train_sents):
        train_data = [[(t,c) for w,t,c in nltk.chunk.tree2conlltags(sent)]
                     for sent in train_sents]
        self.tagger = nltk.UnigramTagger(train_data)
        
    def parse(self, sentence):
        pos_tags = [pos for (word, pos) in sentence]
        tagged_pos_tags = self.tagger.tag(pos_tags)
        chunktags = [chunktag for (pos, chunktag) in tagged_pos_tags]
        conlltags = [(word, pos, chunktag) for ((word, pos), chunktag)
                    in zip(sentence, chunktags)]
        return nltk.chunk.conlltags2tree(conlltags)

In [42]:
test_sents = conll2000.chunked_sents('test.txt', chunk_types=['NP'])
train_sents = conll2000.chunked_sents('train.txt', chunk_types=['NP'])
unigram_chunker = UnigramChunker(train_sents)
print(unigram_chunker.evaluate(test_sents))

ChunkParse score:
    IOB Accuracy:  92.9%%
    Precision:     79.9%%
    Recall:        86.8%%
    F-Measure:     83.2%%


Let's use the unigram tagger to assign a tag to each of the POS tags that appear in the corpus:

In [43]:
postags = sorted(
    set(pos for sent in train_sents for (word, pos) in sent.leaves()))

In [44]:
print(unigram_chunker.tagger.tag(postags))

[('#', 'B-NP'), ('$', 'B-NP'), ("''", 'O'), ('(', 'O'), (')', 'O'), (',', 'O'), ('.', 'O'), (':', 'O'), ('CC', 'O'), ('CD', 'I-NP'), ('DT', 'B-NP'), ('EX', 'B-NP'), ('FW', 'I-NP'), ('IN', 'O'), ('JJ', 'I-NP'), ('JJR', 'B-NP'), ('JJS', 'I-NP'), ('MD', 'O'), ('NN', 'I-NP'), ('NNP', 'I-NP'), ('NNPS', 'I-NP'), ('NNS', 'I-NP'), ('PDT', 'B-NP'), ('POS', 'B-NP'), ('PRP', 'B-NP'), ('PRP$', 'B-NP'), ('RB', 'O'), ('RBR', 'O'), ('RBS', 'B-NP'), ('RP', 'O'), ('SYM', 'O'), ('TO', 'O'), ('UH', 'O'), ('VB', 'O'), ('VBD', 'O'), ('VBG', 'O'), ('VBN', 'O'), ('VBP', 'O'), ('VBZ', 'O'), ('WDT', 'B-NP'), ('WP', 'B-NP'), ('WP$', 'B-NP'), ('WRB', 'O'), ('``', 'O')]


Building a bigram chunker is similar to building a Unigram one. It has a slightly higher performance.

In [48]:
class BigramChunker(nltk.ChunkParserI):
    
    def __init__(self, train_sents):
        train_data = [[(t,c) for w,t,c in nltk.chunk.tree2conlltags(sent)]
                     for sent in train_sents]
        self.tagger = nltk.BigramTagger(train_data)
        
    def parse(self, sentence):
        pos_tags = [pos for (word, pos) in sentence]
        tagged_pos_tags = self.tagger.tag(pos_tags)
        chunktags = [chunktag for (pos, chunktag) in tagged_pos_tags]
        conlltags = [(word, pos, chunktag) for ((word, pos), chunktag)
                    in zip(sentence, chunktags)]
        return nltk.chunk.conlltags2tree(conlltags)

In [46]:
bigram_chunker = BigramChunker(train_sents)

In [47]:
print(bigram_chunker.evaluate(test_sents))

ChunkParse score:
    IOB Accuracy:  93.3%%
    Precision:     82.3%%
    Recall:        86.8%%
    F-Measure:     84.5%%


## Training Classifier-Based Chunkers

In order to maximize the performance of a chunker, we need to consider the content of the words in addition to their POS tags. For this purpose, we'll build a classifier based chunker:

In [83]:
class ConsecutiveNPChunkTagger(nltk.TaggerI):

    def __init__(self, train_sents):
        train_set = []
        for tagged_sent in train_sents:
            untagged_sent = nltk.tag.untag(tagged_sent)
            history = []
            for i, (word, tag) in enumerate(tagged_sent):
                featureset = npchunk_features(untagged_sent, i, history)
                train_set.append( (featureset, tag) )
                history.append(tag)
        self.classifier = nltk.NaiveBayesClassifier.train(
            train_set)

    def tag(self, sentence):
        history = []
        for i, word in enumerate(sentence):
            featureset = npchunk_features(sentence, i, history)
            tag = self.classifier.classify(featureset)
            history.append(tag)
        return zip(sentence, history)

class ConsecutiveNPChunker(nltk.ChunkParserI):
    def __init__(self, train_sents):
        tagged_sents = [[((w,t),c) for (w,t,c) in
                         nltk.chunk.tree2conlltags(sent)]
                        for sent in train_sents]
        self.tagger = ConsecutiveNPChunkTagger(tagged_sents)

    def parse(self, sentence):
        tagged_sents = self.tagger.tag(sentence)
        conlltags = [(w,t,c) for ((w,t),c) in tagged_sents]
        return nltk.chunk.conlltags2tree(conlltags)

In [84]:
# Defining a basic feature extractor
def npchunk_features(sentence, i, history):
    word, pos = sentence[i]
    return {"pos": pos}

The above feature extractor just extracts the POS tag of a token. Thus, it is very similar to the Unigram tagger.

In [85]:
chunker = ConsecutiveNPChunker(train_sents)
print(chunker.evaluate(test_sents))

ChunkParse score:
    IOB Accuracy:  92.9%%
    Precision:     79.9%%
    Recall:        86.8%%
    F-Measure:     83.2%%


We'll now add a feature for the previous part-of-speech tag.

In [86]:
def npchunk_features(sentence, i, history):
    word, pos = sentence[i]
    if i==0:
        prevword, prevpos = "<START>", "<START>"
    else:
        prevword, prevpos = sentence[i-1]
    return {"pos": pos, "prevpos": prevpos}

In [87]:
chunker = ConsecutiveNPChunker(train_sents)
print(chunker.evaluate(test_sents))

ChunkParse score:
    IOB Accuracy:  93.6%%
    Precision:     81.9%%
    Recall:        88.6%%
    F-Measure:     85.1%%


To improve things further, we'll add another feature for the current word:

In [88]:
def npchunk_features(sentence, i, history):
    word, pos = sentence[i]
    if i==0:
        prevword, prevpos = "<START>", "<START>"
    else:
        prevword, prevpos = sentence[i-1]
    return {"pos": pos, "prevpos": prevpos, "word": word}

In [89]:
chunker = ConsecutiveNPChunker(train_sents)
print(chunker.evaluate(test_sents))

ChunkParse score:
    IOB Accuracy:  94.4%%
    Precision:     84.1%%
    Recall:        89.8%%
    F-Measure:     86.9%%


Finally, let's add a few more features, including lookahead features(next pos), pos pairs, and another complex contextual feature `tags-since-dt` which describes the set of all part-of-speech tags that have been encountered since the most recent determiner. 

In [90]:
def npchunk_features(sentence, i, history):
    word, pos = sentence[i]
    if i==0:
        prevword, prevpos = "<START>", "<START>"
    else:
        prevword, prevpos = sentence[i-1]
    if i == len(sentence) - 1:
        nextword, nextpos = "<END>", "<END>"
    else:
        nextword, nextpos = sentence[i+1]
    return {
        "pos": pos,
        "word": word,
        "prevpos": prevpos,
        "nextpos": nextpos,
        "prevpos+pos": "%s+%s" % (prevpos, pos),
        "pos+nextpos": "%s+%s" % (pos, nextpos),
        "tags_since_dt": tags_since_dt(sentence, i)
    }

In [91]:
def tags_since_dt(sentence, i):
    tags = set()
    for word, pos in sentence[:i]:
        if pos == "DT":
            tags = set()
        else:
            tags.add(pos)
    return "+".join(sorted(tags))

In [92]:
chunker = ConsecutiveNPChunker(train_sents)
print(chunker.evaluate(test_sents))

ChunkParse score:
    IOB Accuracy:  95.0%%
    Precision:     85.9%%
    Recall:        90.0%%
    F-Measure:     87.9%%


 ## Recursion in Linguistic Structure
 
 ### Building Nested Structure with Cascaded Chunkers
 
 It is possible to create chunk structures that contain other chunks, by creating a multi-stage chunk grammar with recursive rules. 

In [93]:
grammar = r"""
  NP: {<DT|JJ|NN.*>+}          # Chunk sequences of DT, JJ, NN
  PP: {<IN><NP>}               # Chunk prepositions followed by NP
  VP: {<VB.*><NP|PP|CLAUSE>+$} # Chunk verbs and their arguments
  CLAUSE: {<NP><VP>}           # Chunk NP, VP
  """
cp = nltk.RegexpParser(grammar)
sentence = [("Mary", "NN"), ("saw", "VBD"), ("the", "DT"), ("cat", "NN"),
    ("sit", "VB"), ("on", "IN"), ("the", "DT"), ("mat", "NN")]

In [94]:
print(cp.parse(sentence))

(S
  (NP Mary/NN)
  saw/VBD
  (CLAUSE
    (NP the/DT cat/NN)
    (VP sit/VB (PP on/IN (NP the/DT mat/NN)))))


In the above case the chunk beginning with `VP` was missed by the chunker. We can see that this is the case when a sentence has deper nesting.

In [95]:
sentence = [("John", "NNP"), ("thinks", "VBZ"), ("Mary", "NN"),
    ("saw", "VBD"), ("the", "DT"), ("cat", "NN"), ("sit", "VB"),
    ("on", "IN"), ("the", "DT"), ("mat", "NN")]
print(cp.parse(sentence))

(S
  (NP John/NNP)
  thinks/VBZ
  (NP Mary/NN)
  saw/VBD
  (CLAUSE
    (NP the/DT cat/NN)
    (VP sit/VB (PP on/IN (NP the/DT mat/NN)))))


To solve this, we can specify an argument `loop` to specify the number of times the set of patterns should be run.

In [97]:
cp = nltk.RegexpParser(grammar, loop=3)
print(cp.parse(sentence))

(S
  (CLAUSE
    (NP John/NNP)
    (VP
      thinks/VBZ
      (CLAUSE
        (NP Mary/NN)
        (VP
          saw/VBD
          (CLAUSE
            (NP the/DT cat/NN)
            (VP sit/VB (PP on/IN (NP the/DT mat/NN)))))))))


### Trees

A tree is a set of connected labeled nodes, each reachable by a unique path from a distinguised root note.

In NLTK, we can create trees by giving a node label and a list of children:

In [98]:
tree1 = nltk.Tree("NP", ["Alice"])
print(tree1)

(NP Alice)


In [100]:
tree2 = nltk.Tree("NP", ["the", "rabbit"])
print(tree2)

(NP the rabbit)


We can incorporate trees into other trees as follows:

In [101]:
tree3 = nltk.Tree("VP", ["chased", tree2])
tree4 = nltk.Tree("S", [tree1, tree3])
print(tree4)

(S (NP Alice) (VP chased (NP the rabbit)))


Here are some of the methods available for the tree objects:

In [102]:
print(tree4[1])

(VP chased (NP the rabbit))


Here are some methods available for tree objects:

In [103]:
print(tree4[1])

(VP chased (NP the rabbit))


In [104]:
tree4[1].label()

'VP'

In [105]:
tree4.leaves()

['Alice', 'chased', 'the', 'rabbit']

In [106]:
tree4[1][1][1]

'rabbit'

In [107]:
# To print a graphical representation of a tree
# tree.draw()

### Tree Traversal

In [108]:
def traverse(t):
    try:
        t.label()
    except AttributeError:
        print(t, end=" ")
    else:
        print("(", t.label(), end=" ")
        for child in t:
            traverse(child)
            print(")", end=" ")

In [111]:
t = nltk.Tree.fromstring('(S (NP Alice) (VP chased (NP the rabbit)))')

In [112]:
traverse(t)

( S ( NP Alice ) ) ( VP chased ) ( NP the ) rabbit ) ) ) 

## Named Entity Recognition

NLTK provides a built in classifier that has been trained to recognize named entities:

In [114]:
sent = nltk.corpus.treebank.tagged_sents()[22]
print(nltk.ne_chunk(sent, binary=True))

(S
  The/DT
  (NE U.S./NNP)
  is/VBZ
  one/CD
  of/IN
  the/DT
  few/JJ
  industrialized/VBN
  nations/NNS
  that/WDT
  *T*-7/-NONE-
  does/VBZ
  n't/RB
  have/VB
  a/DT
  higher/JJR
  standard/NN
  of/IN
  regulation/NN
  for/IN
  the/DT
  smooth/JJ
  ,/,
  needle-like/JJ
  fibers/NNS
  such/JJ
  as/IN
  crocidolite/NN
  that/WDT
  *T*-1/-NONE-
  are/VBP
  classified/VBN
  *-5/-NONE-
  as/IN
  amphobiles/NNS
  ,/,
  according/VBG
  to/TO
  (NE Brooke/NNP)
  T./NNP
  Mossman/NNP
  ,/,
  a/DT
  professor/NN
  of/IN
  pathlogy/NN
  at/IN
  the/DT
  (NE University/NNP)
  of/IN
  (NE Vermont/NNP College/NNP)
  of/IN
  (NE Medicine/NNP)
  ./.)


## Relation Extraction

Let's look at relations between specified types of named entity. 
In the following example, we'll search for strings of the for `(X, a, Y)` where X and Y are named entities, and a is a string of words that intervenes between them.

In [118]:
IN = re.compile(r".*\bin\b(?!\b.ing)")

In [120]:
for doc in nltk.corpus.ieer.parsed_docs("NYT_19980315"):
    for rel in nltk.sem.extract_rels("ORG", "LOC", doc, corpus="ieer", pattern=IN):
        print(nltk.sem.rtuple(rel))

[ORG: 'WHYY'] 'in' [LOC: 'Philadelphia']
[ORG: 'McGlashan &AMP; Sarrail'] 'firm in' [LOC: 'San Mateo']
[ORG: 'Freedom Forum'] 'in' [LOC: 'Arlington']
[ORG: 'Brookings Institution'] ', the research group in' [LOC: 'Washington']
[ORG: 'Idealab'] ', a self-described business incubator based in' [LOC: 'Los Angeles']
[ORG: 'Open Text'] ', based in' [LOC: 'Waterloo']
[ORG: 'WGBH'] 'in' [LOC: 'Boston']
[ORG: 'Bastille Opera'] 'in' [LOC: 'Paris']
[ORG: 'Omnicom'] 'in' [LOC: 'New York']
[ORG: 'DDB Needham'] 'in' [LOC: 'New York']
[ORG: 'Kaplan Thaler Group'] 'in' [LOC: 'New York']
[ORG: 'BBDO South'] 'in' [LOC: 'Atlanta']
[ORG: 'Georgia-Pacific'] 'in' [LOC: 'Atlanta']


Further Reading: https://www.nltk.org/book/ch07.html#saw-vbd